# Part 0: Dataloader and Visualizations

In [16]:
import numpy as np
from PIL import Image
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import wandb

from voc_dataset import VOCDataset
from utils import *
%load_ext autoreload
%autoreload 2
USE_WANDB = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Q0.1: Editing the Dataloader
The first part of the assignment involves editing the dataloader so that we can access bounding-box proposals as well as the ground-truth bounding boxes. The ground truth bounding box can be accessed through the VOC Dataset annotations itself and we have completed this part for you in the starter code. 

Unsupervised bounding box proposals are obtained through methods such as [Selective Search](https://ivi.fnwi.uva.nl/isis/publications/2013/UijlingsIJCV2013/UijlingsIJCV2013.pdf). Since Selective Search is slow to run on each image, we have pre-computed the bounding box proposals for you (you downloaded this in the data preparation step).

Your task is to change the dataloader to obtain the proposed bounding boxes for each image. Feel free to experiment with the data in the files to figure out the number of proposals per image, their scores, etc. Returning a dictionary would be convenient here. For the bounding boxes, using the relative positions is usually a better idea since they are invariant to changes in the size of the image.

In [17]:
dataset = VOCDataset('trainval', top_n=10)

**Q0.1**: Load the image corresponding to index 2020 and print the GT labels associated with it.

**Hint**: items at a particular index can be accesed by usual indexing notation (dataset[idx])

In [18]:
# TODO: get the image information from index 2020
idx = 2020
dataset.roi_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'boxScores', 'boxes', 'images'])

## Q0.2 and Q0.3: Wandb Logging
First, let's initialize a Weights and Biases project. 

In [19]:
USE_WANDB = True
if USE_WANDB:
    wandb.init(project="vlr-hw1", reinit=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: khjune29 (cvl2023). Use `wandb login --relogin` to force relogin


**Q0.2**: Complete this block for overlaying the ground truth box on an image.

**Hint**: convert the image tensor to a PIL image and plot it (check `utils.py` for helper functions). You can use [this](https://docs.wandb.ai/library/log) as a reference for logging syntax.

In [20]:
class_id_to_label = dict(enumerate(dataset.CLASS_NAMES))

# TODO: load the GT information corresponding to index 2020.
print(dataset[idx].keys())
original_image = dataset[idx]['image']
gt_labels = dataset[idx]['gt_classes']
gt_boxes = dataset[idx]['gt_boxes']

img = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(gt_labels, gt_boxes),
        "class_labels": class_id_to_label,
    },
})

# TODO: log the GT bounding box


dict_keys(['image', 'label', 'wgt', 'rois', 'gt_boxes', 'gt_classes'])


**Q0.3**: Visualize the top 10 bounding proposals corresponding to index 2020.

**Hint**: Check the `get_box_data` function in `utils.py` and understand how it is being used. 

In [23]:
# TODO: plot top ten proposals (of bounding boxes)
idx=2023

print(dataset.roi_data['boxes'][0][idx][:10])
imgs, boxes = [], []
for idx in range(2023, 2026):
    rois = dataset[idx]['rois']
    imgs.append(tensor_to_PIL(dataset[idx]['image']))
    boxes.append({
        "proposals": {
            "box_data": [
                {
                    "position": {
                        "minX": bbox[0],
                        "minY": bbox[1], #top(min row)
                        "maxX": bbox[2],
                        "maxY": bbox[3], #bottom(max row)
                        },
                    "class_id": 0
                }
                for bbox in rois]
        },
    })
    
wdb_img = [wandb.Image(img, boxes=box) for img, box in zip(imgs, boxes)]
wandb.log({"region proposals": wdb_img})


[[  1 186 321 351]
 [ 92   1 418 500]
 [  1   1 366 395]
 [  1 185 117 353]
 [  1 172 374 500]
 [  1   1 302 352]
 [  1 172 363 397]
 [ 98   1 315 279]
 [ 92   1 365 397]
 [132   1 305 204]]
